In [1]:
import laspy as laspy
import h5py
from geojson import Point, Feature, FeatureCollection, dump

In [2]:
import os
import numpy as N
import math

In [3]:
def find_label(file):
    obj = file.split('_')[0]
    if obj == 'truck':
        return 0
    elif obj == 'jcb':
        return 1
    else:
        return 2

In [4]:
def create_normalized_blocks(path, num_points,rgb):
    files = os.listdir(path)
    print(len(files))
    source_file = []
    label = []
    if rgb:
        normalized_blocks = N.zeros((0,num_points,6))
        block_min_array = N.zeros((0,6))
        block_max_array = N.zeros((0,6))
    else:
        normalized_blocks = N.zeros((0,num_points,3))
        block_min_array = N.zeros((0,3))
        block_max_array = N.zeros((0,3))
    for file in files:
        inFile = laspy.file.File(os.path.join(path,file), mode = 'r')
        unscaled_points = N.vstack([inFile.X, inFile.Y, inFile.Z]).transpose()
        if(len(list(unscaled_points))<num_points):
            print(file)
        if(len(list(unscaled_points))>=num_points):
            
            head = inFile.header
            scale = head.scale
            offset = head.offset
            points = unscaled_points * scale + offset
            if rgb:
                unscaled_rgb_vals = N.vstack([inFile.Red, inFile.Green, inFile.Blue]).transpose()
                rgb_vals = unscaled_rgb_vals/256
                points = N.concatenate((points,rgb_vals),axis=1)
               
            #block_min = head.min
            #block_max = head.max
            
            #sample
            random_indexes = N.random.randint(0, high=len(points)-1, size=num_points)
            block = points[random_indexes]
            
            #normalize
            block_min = N.min(block ,axis = 0)
            block_max = N.max(block ,axis = 0)
            
            points_sub = block - block_min
            diff = N.array(block_max) - N.array(block_min)
            n_block = points_sub/diff
            normalized_blocks = N.append(normalized_blocks,[n_block],axis=0)
            block_min_array = N.append(block_min_array,[block_min],axis=0)
            block_max_array = N.append(block_max_array,[block_max],axis=0)
            source_file.append(file)            
            label.append(find_label(file))  
                           
    return normalized_blocks,source_file,block_min_array,block_max_array,label   

In [6]:
# file has string data!!!!

def write_block_to_h5(filename, data, file, block_min_array, block_max_array,label):
    hf = h5py.File(filename, 'w')
    hf.create_dataset('data', data=data)
    asciiList = [n.encode("ascii", "ignore") for n in file]
    string_type = h5py.special_dtype(vlen=bytes)
    hf.create_dataset('source_file', shape = (len(asciiList),1), data = asciiList, dtype=string_type)
    hf.create_dataset('min',data = block_min_array)
    hf.create_dataset('max',data = block_max_array)
    hf.create_dataset('label',data = label)
    hf.close()

In [7]:
def loadBlockData(h5_filename):
    f = h5py.File(h5_filename)
    data = f['data'][:]
    files = f['source_file'][:]
    return (data, files)


In [12]:
def find_block(filename, query_x, query_y):
    f = h5py.File(filename)
    files = f['source_file'][:]
    min_data = f['min'][:]
    max_data = f['max'][:]
    feature_collection = []
    for i in range(min_data.shape[0]):
        if(min_data[i][0]<query_x<max_data[i][0] and min_data[i][1]<query_y<max_data[i][1]):
            print(files[i][0].decode("utf-8"))
            point1 = Point((min_data[i][0],min_data[i][1],min_data[i][2]))
            point2 = Point((max_data[i][0],max_data[i][1],max_data[i][2]))
                
            f1 = Feature(geometry=point1)
            feature_collection.append(f1)
            f2 = Feature(geometry=point2)
            feature_collection.append(f2)

            fc = FeatureCollection(feature_collection)
            with open('min_max.geojson', 'w') as f:
                dump(fc, f)    
            return files[i][0].decode("utf-8")

In [5]:
def split_data(normalized_blocks, source_file, block_min_array, block_max_array, label):
    length = len(label)
    train_num = 360
    eval_num = 48
    #eval_num = int(0.15*length)
    test_num = length - (train_num + eval_num)
    
    print(train_num,eval_num,test_num)
    
    #train data
    t_data = normalized_blocks[0:train_num,:]
    t_file = source_file[0:(train_num)]
    t_label = label[0:(train_num)]
    t_block_min_array = block_min_array[0:(train_num),:]
    t_block_max_array = block_max_array[0:(train_num),:]
    #write_block_to_h5('./training_files/train_rot_20.h5',t_data,t_file,t_block_min_array, t_block_max_array,t_label)
    
    #evL data
    e_data = normalized_blocks[train_num:train_num+eval_num,:]
    e_file = source_file[train_num:train_num+eval_num]
    e_label = label[train_num:train_num+eval_num]
    e_block_min_array = block_min_array[train_num:train_num+eval_num,:]
    e_block_max_array = block_max_array[train_num:train_num+eval_num,:]
    #write_block_to_h5('./training_files/eval_rot_20.h5',e_data,e_file,e_block_min_array, e_block_max_array,e_label)
    
    #TEST data
    
    test_data = normalized_blocks[train_num+eval_num:,:]
    test_file = source_file[train_num+eval_num:]
    test_label = label[train_num+eval_num:]
    test_block_min_array = block_min_array[train_num+eval_num:,:]
    test_block_max_array = block_max_array[train_num+eval_num:,:]
    #write_block_to_h5('./training_files/test_rot_20.h5',test_data,test_file,test_block_min_array, test_block_max_array,test_label)
        
    return t_data, t_file, t_label, e_data, e_file, e_label, test_data, test_file, test_label

In [8]:
num_points = 1024
rgb=False
#path = "/opt/datasets/object_detection/pointnet/files/train_data_3d"
path0 = "/home/srividya/github/pointnet/training_files/rotated_las_20/trucks"
normalized_blocks, source_file, block_min_array, block_max_array, label = create_normalized_blocks(path0,num_points,rgb)
td0,tf0,tl0,ed0,ef0,el0,test_data0, test_file0,test_label0 = split_data(normalized_blocks, source_file, block_min_array, block_max_array, label)

path2 = "/home/srividya/github/pointnet/training_files/rotated_las_20/bg"
normalized_blocks, source_file, block_min_array, block_max_array, label = create_normalized_blocks(path2,num_points,rgb)
td2,tf2,tl2,ed2,ef2,el2,test_data2, test_file2,test_label2 = split_data(normalized_blocks, source_file, block_min_array, block_max_array, label)

450
360 48 42
540
360 48 132


In [9]:
train_file = []
eval_file = []
test_file = []
train_label = []
test_label = []
eval_label = []

In [10]:

tdata = N.concatenate((td0, td2))
edata = N.concatenate((ed0, ed2))
testdata = N.concatenate((test_data0, test_data2))
train_file.extend(tf0)
train_file.extend(tf2)
eval_file.extend(ef0)
eval_file.extend(ef2)
test_file.extend(test_file0)
test_file.extend(test_file2)
train_label.extend(tl0)
train_label.extend(tl2)
eval_label.extend(el0)
eval_label.extend(el2)
test_label.extend(test_label0)
test_label.extend(test_label2)


In [15]:
test_label = [1 if label==2 else label for label in test_label]
eval_label = [1 if label==2 else label for label in eval_label]
train_label = [1 if label==2 else label for label in train_label]

In [16]:
write_block_to_h5('./training_files/train_rot_20_2class.h5',tdata,train_file,[],[],train_label)
write_block_to_h5('./training_files/eval_rot_20_2class.h5',edata,eval_file,[],[],eval_label)
write_block_to_h5('./training_files/test_rot_20_2class.h5',testdata,test_file,[],[],test_label)

In [14]:
print(len(train_label))

720


## Augmentation


In [70]:
def rotate_data(path, step):
    files = os.listdir(path)
    print(len(files))
    for file in files:
        inFile = laspy.file.File(os.path.join(path,file), mode = 'r')
        unscaled_points = N.vstack([inFile.X, inFile.Y, inFile.Z]).transpose()
            
        head = inFile.header
        scale = head.scale
        offset = head.offset
        points = unscaled_points * scale + offset
        
        for z_deg in range(0,360,step):
            #print(z_deg)
            z = z_deg * N.pi/180
            cosz = math.cos(z)
            sinz = math.sin(z)

            rotation_mat = N.array(
                    [[cosz, -sinz, 0],
                     [sinz, cosz, 0],
                     [0, 0, 1]])
            
            rotated_points = N.dot(points, rotation_mat)
                        
            scaled_pts = (rotated_points - offset) / scale
            
            header_h = laspy.header.Header()
            op_file_name = file.split('.')[0]+'_'+str(z_deg)+'.las'
            outfile = laspy.file.File(os.path.join('./training_files/rotated_las_20',op_file_name), mode="w", header=head)
            outfile.X = scaled_pts[:,0]
            outfile.Y = scaled_pts[:,1]
            outfile.Z = scaled_pts[:,2]           
            outfile.close()    

In [71]:
path = "/opt/datasets/object_detection/pointnet/files/train_data_3d"
step = 20
rotate_data(path, step)

109


# Nearest Neighbor

In [10]:
block_df = P.read_pickle('feature_0.pkl')
#feature = block_df['feature']
feature_array = N.load('feature_0.npy')
print(feature_array.shape)

(3252, 1024)


In [15]:
query_x = 5066.69
query_y = 7849.21

from sklearn.neighbors import KDTree
#tree = KDTree(block_df['feature'].as_matrix())  
tree = KDTree(feature_array)
block_file = find_block('block_rmz.h5',query_x, query_y)
index = block_df['file'] == block_file
dist, ind = tree.query(feature_array[index].reshape(1,-1), k=10)                

truck_3.las


/mnt/research-disk/virtualenvs/tf-cpu1/lib/python3.6/site-packages/ipykernel_launcher.py:2: H5pyDeprecationWarning: The default file mode will change to 'r' (read-only) in h5py 3.0. To suppress this warning, pass the mode you need to h5py.File(), or set the global default h5.get_config().default_file_mode, or set the environment variable H5PY_DEFAULT_READONLY=1. Available modes are: 'r', 'r+', 'w', 'w-'/'x', 'a'. See the docs for details.
  


In [16]:
print(ind)

[[1928  748  719 2973 1930 2821 1411 1373 2395 2403]]


In [17]:
nn_files = list(block_df['file'][ind[0]])
print(nn_files)

['truck_3.las', 'truck_4.las', 'split_5-14-68.las', 'split_5-45-33.las', 'truck_1.las', 'split_5-17-80.las', 'truck_2.las', 'split_5-58-41.las', 'split_5-41-28.las', 'split_5-27-45.las']


In [21]:

feature_collection = []
nn_files = list(block_df['file'][ind[0]])
idx=0
for file in nn_files:
    inFile = laspy.file.File(os.path.join(ds.path(),file), mode = 'r')
    unscaled_points = N.vstack([inFile.X, inFile.Y, inFile.Z]).transpose()
    head = inFile.header
    scale = head.scale
    offset = head.offset
    points = unscaled_points * scale + offset
    center = N.mean(points,axis=0)
    idx=idx+1
    point = Point(list(center))    
    feature = Feature(geometry=point,id=idx)
    feature_collection.append(feature)
    
fc = FeatureCollection(feature_collection)
with open('point_trucks.geojson', 'w') as f:
    dump(fc, f)    

In [45]:
print(center)

[4983.84012703 7768.05703727  591.26713637]


### Visualize point cloud

In [ ]:
from utils import pc_util
import imageio

for i in range(normalized_blocks.shape[0]):
    img = pc_util.point_cloud_three_views(normalized_blocks[i,:,:])
    filename = os.path.join('./images',source_file[i].split('.')[0]+'.jpg')
    imageio.imwrite(filename,img)